In [6]:
import json
import requests
from tqdm.notebook import tqdm
import time

In [2]:
api_token = 'c760c2344bad4f24b4344c2d113b8894'
api_url = 'https://api.labs.cognitive.microsoft.com/academic/v1.0/evaluate'
headers = {'Ocp-Apim-Subscription-Key': api_token}

In [3]:
def reformat_result(result):
    fos_list = []
    if 'entities' in result:
        for entity in result['entities']:
            fos = {}
            fos['id'] = entity['Id']
            fos['name'] = entity['FN']
            fos['displayed_name'] = entity['DFN']
            fos['level'] = entity['FL']
            
            if 'FP' in entity:
                fos['parent_ids'] = [parent['FId'] for parent in entity['FP']]
            else:
                fos['parent_ids'] = []
            
            fos_list.append(fos)
    
    return fos_list

In [28]:
query = {'expr': 'And(Ty=\'6\',FN=\'machine learning\')',
         'model': 'latest',
         'attributes': 'Id,FN,DFN,FL,FP.FId',
         'count': '10',
         'offset': '0',
        }

r = requests.get(api_url, params=query, headers=headers)

fos = reformat_result(r.json())[0]
complete_fos_dict = {fos['id']: fos}
current_fos_dict = {fos['id']: fos}

In [29]:
fos

{'id': 119857082,
 'name': 'machine learning',
 'displayed_name': 'Machine learning',
 'level': 1,
 'parent_ids': [41008148]}

In [30]:
query = {'expr': 'And(Ty=\'6\',Composite(FP.FId=119857082))',
         'model': 'latest',
         'attributes': 'Id,FN,DFN,FL,FP.FId',
         'count': '1000',
         'offset': '0',
        }
r = requests.get(api_url, params=query, headers=headers)
len(reformat_result(r.json()))

798

In [31]:
while current_fos_dict:
    last_fos_list = current_fos_dict.values()
    current_fos_dict = {}
    
    for fos in tqdm(last_fos_list):
        time.sleep(1)
        query = {'expr': 'And(Ty=\'6\',Composite(FP.FId={:d}))'.format(fos['id']),
                 'model': 'latest',
                 'attributes': 'Id,FN,DFN,FL,FP.FId',
                 'count': '1000',
                 'offset': '0',
                }
        r = requests.get(api_url, params=query, headers=headers)
        
        new_fos_list = reformat_result(r.json())
        if len(new_fos_list) == 1000:
            print('need to deal with offset')
                
        for new_fos in new_fos_list:
            if new_fos['id'] not in complete_fos_dict and new_fos['id'] not in current_fos_dict:
                current_fos_dict[new_fos['id']] = new_fos
                complete_fos_dict[new_fos['id']] = new_fos
                
    print('{:d} fos added'.format(len(current_fos_dict)))


798 fos added



1662 fos added



716 fos added



605 fos added



0 fos added


In [32]:
len(complete_fos_dict)

3782

In [33]:
with open('./datasets/fos_hierarchy.json', 'w') as f:
    json.dump(complete_fos_dict, f)